# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinu

In [38]:
%cd '/content/drive/MyDrive/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/'
/content


In [39]:
!unzip /content/data.zip

unzip:  cannot find or open /content/data.zip, /content/data.zip.zip or /content/data.zip.ZIP.


# *Importing libraries*

In [2]:
import json
import wandb
import pandas as pd
import pickle
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")

train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])

# Code for Trans-Literared Text using IndicTrans Library

In [4]:
!git clone https://github.com/libindic/indic-trans.git
%cd /content/indic-trans
!pip install -r requirements.txt
!pip install .

Cloning into 'indic-trans'...
remote: Enumerating objects: 2225, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2225 (delta 8), reused 10 (delta 3), pack-reused 2206 (from 1)
Receiving objects: 100% (2225/2225), 516.51 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (1102/1102), done.
Updating files: 100% (719/719), done.
/content/indic-trans
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 6.0 MB/s eta 0:00:00
Processing /content/indic-trans
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for indictrans: filename=indictrans-1.2.3-cp310-cp310-linux_x86_64.whl size=337821303 sha256=fefeaa926673133cdfd4431bae311f26ca31612b662d12b1540e234724f51fe7
  Stored in directory: /root/.cache/pip/wheels/3e/c9/43/39c5aaa9a570043089bac219a37343294fa5b47f1350dea53a
Successfully buil

In [5]:
!indictrans --h

usage: indictrans [-h] [-v] [-s] [-t] [-b] [-m | -r] [-i] [-o]

Transliterator for Indian Languages including English

options:
  -h, --help          show this help message and exit
  -v, --version       show program's version number and exit
  -s , --source       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -t , --target       select language (3 letter ISO-639 code) {hin, guj, pan, ben, mal, kan, tam,
                      tel, ori, eng, mar, nep, bod, kok, asm, urd}
  -b, --build-lookup  build lookup to fasten transliteration
  -m, --ml            use ML system for transliteration
  -r, --rb            use rule-based system for transliteration
  -i , --input        <input-file>
  -o , --output       <output-file>


In [6]:
%cd /content/

/content


In [7]:
from indictrans import Transliterator
trn = Transliterator(source='hin', target='eng', build_lookup=True)
hin = """कांग्रेस पार्टी अध्यक्ष सोनिया गांधी, तमिलनाडु की मुख्यमंत्री
     जयललिता और रिज़र्व बैंक के गवर्नर रघुराम राजन के बीच एक समानता
     है. ये सभी अलग-अलग कारणों से भारतीय जनता पार्टी के राज्यसभा सांसद
     सुब्रमण्यम स्वामी के निशाने पर हैं. उनके जयललिता और सोनिया गांधी के
     पीछे पड़ने का कारण कथित भ्रष्टाचार है."""
eng = trn.transform(hin)
print(eng)

congress party adhyaksh sonia gandhi, tamilnadu kii mukhyamantri
     jayalalita or reserve bank ke governor raghuram rajan ke bich ek samanta
     he. ye sabhi alag-alag kaarnon se bhartiya janata party ke rajyasabha saansad
     subramanyam swami ke nishane par hai. unke jayalalita or sonia gandhi ke
     peeche padane kaa kaaran kathith bhrashtachar he.


In [8]:
trn_en_hin = Transliterator(source='eng', target='hin', build_lookup=True)

In [9]:
import pickle
with open('/content/rh-code-mixed-2.pkl', 'rb') as f:
    related_words = pickle.load(f)

In [10]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [11]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [12]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [13]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [14]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
})

## Transliterated Word language with Devanagri Hindi scipt and english words using language tags Approach

In [16]:
def transliterated_preprocess(sentences, related_words):
    texts = sentences['clean_text']
    processed_texts = []

    for text in texts:
        words = text.split()
        process_text = []

        for word in words:
            if related_words[word].get('language') == 'HI':
                process_text.append(trn_en_hin.transform(related_words[word]['corrected_word']))
            else:
              if related_words[word].get('corrected_word') != None:
                process_text.append(related_words[word]['corrected_word'])
              else:
                process_text.append(word)

        # Join the list back into a string
        processed_texts.append(" ".join(process_text))

    return {"transliterated_skip_eng": processed_texts}

dataset = dataset.map(
    transliterated_preprocess,
    fn_kwargs={'related_words': related_words},
    batched=True
)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [17]:
print(dataset['train'][0]['transliterated_skip_eng'])

hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है


In [18]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'transliterated_skip_eng': 'hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है'}

In [19]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'transliterated_skip_eng': 'hindu rastra के रुझाँ मिया भाई को कस्ट है चलो hindu इस bat पे mast है'}

In [20]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'transliterated_skip_eng': 'जय hind , in नमूनों को bat भी करनी नहीं अति है , इनको pension चाहिए'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [21]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


In [22]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['transliterated_skip_eng'],max_length=97,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [23]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  21042,  22022,  10569,  12213,    880,  14070,
          13841,  21042,  96597,  83874,  70219,    867,  52768,  34315,    889,
          22078,  33555,  13220,  10345,  11519,  57203,    888,  19741,  72109,
          10826,  21042,    880,  14070,  45753, 103860,  11357,  10113,  10412,
          49545,  11554,    888,  19741,  28960,  14080,  17798,    880,  11714,
          52884,  10412, 100906,  11072,    898,  11714,  52884,    866,  51140,
          41607,  27640,  53744,    880,  14070,  45753,  42263,  10914,  38207,
            867,  11714,  18406,  16791,  10569,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [24]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'तो', 'कुछ', 'है', 'पर', 'त', '##ु', '##म', 'तो', 'पूरी', 'दुनिया', 'मई', 'ग', '##ंद', '##गी', 'म', '##चा', 'रहे', 'हो', 'or', 'bat', 'रही', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##वार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'नहीं', 'है', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [25]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['transliterated_skip_eng']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है
tokenize:              ['hin', '##du', 'तो', 'कुछ', 'है', 'पर', 'त', '##ु', '##म', 'तो', 'पूरी', 'दुनिया', 'मई', 'ग', '##ंद', '##गी', 'म', '##चा', 'रहे', 'हो', 'or', 'bat', 'रही', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##वार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'नहीं', 'है']
convert_tokens_to_ids: [19911, 11460, 21042, 22022, 10569, 12213, 880, 14070, 13841, 21042, 96597, 83874, 70219, 867, 52768, 34315, 889, 22078, 33555, 13220, 10345, 11519, 57203, 888, 19741, 72109, 10826, 21042, 880, 14070, 45753, 103860, 11357, 10113, 10412, 49545, 11554, 888, 19741, 28960, 14080, 17798, 880, 11714,

# *Calculating Weights for Labels with Imbalanced Dataset*

In [26]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [27]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [28]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'recall_positive': recall_positive,
        'recall_negative': recall_negative,
        'f1_weighted': f1,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_positive': f1_positive,
        'f1_negative': f1_negative
    }

In [68]:
!export CUDA_LAUNCH_BLOCKING=1

In [52]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)


arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_skip_eng",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [36]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [53]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_skip_eng/log.jsonl"))

In [54]:
# train the model
wandb.init(project="rh3", name="Hing_mBERT_skip_eng2")
trainer.train()

eval/accuracy,█▇▁█
eval/f1_macro,▅█▁▅
eval/f1_micro,█▇▁█
eval/f1_negative,█▇▁█
eval/f1_positive,▁█▇▁
eval/f1_weighted,██▁█
eval/loss,█▃▁█
eval/precision_weighted,▁▅█▁
eval/recall_macro,▁█▅▁
eval/recall_micro,█▇▁█
eval/recall_negative,█▅▁█


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.685362,0.757101,0.672043,0.757101,0.757101,0.502111,0.008097,0.996124,0.656890,0.757101,0.438663,0.015873,0.861453
2,No log,0.643570,0.525955,0.731712,0.525955,0.525955,0.614289,0.785425,0.443152,0.552124,0.525955,0.515639,0.444954,0.586325
3,No log,0.581321,0.765916,0.769526,0.765916,0.765916,0.688481,0.538462,0.838501,0.767628,0.765916,0.685617,0.526733,0.844502
4,No log,0.588646,0.780607,0.776499,0.780607,0.780607,0.691280,0.518219,0.864341,0.778391,0.780607,0.694964,0.533333,0.856594
5,No log,0.685285,0.796278,0.784306,0.796278,0.796278,0.687833,0.477733,0.897933,0.787994,0.796278,0.700684,0.531532,0.869837
6,No log,0.667890,0.799216,0.790128,0.799216,0.799216,0.700797,0.510121,0.891473,0.793432,0.799216,0.711042,0.551422,0.870662
7,No log,0.698501,0.687561,0.782684,0.687561,0.687561,0.713986,0.765182,0.662791,0.709482,0.687561,0.652575,0.542324,0.762825
8,No log,0.769832,0.792360,0.784461,0.792360,0.792360,0.696275,0.510121,0.882429,0.787622,0.792360,0.704378,0.543103,0.865653


TrainOutput(global_step=384, training_loss=0.514752467473348, metrics={'train_runtime': 599.7881, 'train_samples_per_second': 51.018, 'train_steps_per_second': 0.8, 'train_loss': 0.514752467473348, 'epoch': 8.0})

In [55]:
mbert_results = trainer.evaluate()

In [56]:
print(mbert_results)

{'eval_loss': 0.6678898334503174, 'eval_accuracy': 0.7992164544564153, 'eval_precision_weighted': 0.7901276643024233, 'eval_recall_weighted': 0.7992164544564153, 'eval_recall_micro': 0.7992164544564153, 'eval_recall_macro': 0.7007971628534664, 'eval_recall_positive': 0.5101214574898786, 'eval_recall_negative': 0.8914728682170543, 'eval_f1_weighted': 0.7934319856902837, 'eval_f1_micro': 0.7992164544564153, 'eval_f1_macro': 0.7110423900213296, 'eval_f1_positive': 0.5514223194748359, 'eval_f1_negative': 0.8706624605678234, 'eval_runtime': 5.3326, 'eval_samples_per_second': 191.464, 'eval_steps_per_second': 12.002, 'epoch': 8.0}


# **Hing-MBERT Test results**

In [57]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [58]:
test_results_mbert

PredictionOutput(predictions=array([[ 1.7165234 , -1.3934427 ],
       [ 1.2097842 , -0.80548924],
       [ 1.857489  , -1.4731256 ],
       ...,
       [-0.4700326 ,  0.694087  ],
       [-1.3955095 ,  1.5566144 ],
       [ 0.29391816, -0.13080703]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.6665571331977844, 'test_accuracy': 0.7806072477962782, 'test_precision_weighted': 0.7726438022805766, 'test_recall_weighted': 0.7806072477962782, 'test_recall_micro': 0.7806072477962782, 'test_recall_macro': 0.681631777715009, 'test_recall_positive': 0.4898785425101215, 'test_recall_negative': 0.8733850129198967, 'test_f1_weighted': 0.7759649696198143, 'test_f1_micro': 0.7806072477962782, 'test_f1_macro': 0.6885906625127993, 'test_f1_positive': 0.5193133047210301, 'test_f1_negative': 0.8578680203045685, 'test_runtime': 5.3869, 'test_samples_per_second': 189.532, 'test_steps_per_second': 11.881})

In [59]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 0])

In [60]:
test_df['skip_eng_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [61]:
print(test_results_mbert.metrics)

{'test_loss': 0.6665571331977844, 'test_accuracy': 0.7806072477962782, 'test_precision_weighted': 0.7726438022805766, 'test_recall_weighted': 0.7806072477962782, 'test_recall_micro': 0.7806072477962782, 'test_recall_macro': 0.681631777715009, 'test_recall_positive': 0.4898785425101215, 'test_recall_negative': 0.8733850129198967, 'test_f1_weighted': 0.7759649696198143, 'test_f1_micro': 0.7806072477962782, 'test_f1_macro': 0.6885906625127993, 'test_f1_positive': 0.5193133047210301, 'test_f1_negative': 0.8578680203045685, 'test_runtime': 5.3869, 'test_samples_per_second': 189.532, 'test_steps_per_second': 11.881}


# **Hing BERT Training**

In [46]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [47]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['transliterated_skip_eng'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [62]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_skip_eng",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_skip_eng/log.jsonl"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [63]:
# train the model
wandb.init(project="rh3", name="Hing_Bert_skip_eng2")
bert_trainer.train()

eval/accuracy,▇▁▇███▅██
eval/f1_macro,▁▃▇███▆██
eval/f1_micro,▇▁▇███▅██
eval/f1_negative,█▁▇███▅██
eval/f1_positive,▁▇███████
eval/f1_weighted,▄▁▇███▆██
eval/loss,▅▃▁▁▅▄▅█▄
eval/precision_weighted,▁▅▇▇█████
eval/recall_macro,▁▅▇▇▇██▇█
eval/recall_micro,▇▁▇███▅██
eval/recall_negative,█▁▆▆▇▇▄▇▇


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.692414,0.658178,0.637766,0.658178,0.658178,0.505780,0.210526,0.801034,0.647119,0.658178,0.504973,0.229581,0.780365
2,No log,0.687698,0.607248,0.692887,0.607248,0.607248,0.586587,0.546559,0.626615,0.633696,0.607248,0.554949,0.402385,0.707513
3,No log,0.667415,0.624878,0.725149,0.624878,0.624878,0.631294,0.643725,0.618863,0.651311,0.624878,0.584015,0.453638,0.714392
4,No log,0.618132,0.702253,0.739111,0.702253,0.702253,0.654762,0.562753,0.746770,0.715790,0.702253,0.634722,0.477663,0.791781
5,No log,0.621142,0.636631,0.732028,0.636631,0.636631,0.641802,0.651822,0.631783,0.662001,0.636631,0.594814,0.464646,0.724981
6,No log,0.612737,0.710088,0.748178,0.710088,0.710088,0.668199,0.587045,0.749354,0.723686,0.710088,0.645792,0.494881,0.796703
7,No log,0.613501,0.668952,0.737116,0.668952,0.668952,0.652094,0.619433,0.684755,0.689746,0.668952,0.616691,0.475155,0.758226
8,No log,0.626740,0.680705,0.743147,0.680705,0.680705,0.661224,0.623482,0.698966,0.700085,0.680705,0.627135,0.485804,0.768466


TrainOutput(global_step=384, training_loss=0.6191465854644775, metrics={'train_runtime': 506.5075, 'train_samples_per_second': 60.414, 'train_steps_per_second': 0.948, 'train_loss': 0.6191465854644775, 'epoch': 8.0})

In [64]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.6127367615699768, 'eval_accuracy': 0.7100881488736532, 'eval_precision_weighted': 0.7481777049557625, 'eval_recall_weighted': 0.7100881488736532, 'eval_recall_micro': 0.7100881488736532, 'eval_recall_macro': 0.6681992697904571, 'eval_recall_positive': 0.5870445344129555, 'eval_recall_negative': 0.7493540051679587, 'eval_f1_weighted': 0.7236864314680683, 'eval_f1_micro': 0.7100881488736532, 'eval_f1_macro': 0.6457919213891911, 'eval_f1_positive': 0.4948805460750853, 'eval_f1_negative': 0.7967032967032968, 'eval_runtime': 5.3637, 'eval_samples_per_second': 190.353, 'eval_steps_per_second': 11.932, 'epoch': 8.0}


# **Test results for Hing-BERT**

In [65]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['skip_eng_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.6173535585403442, 'test_accuracy': 0.7071498530852106, 'test_precision_weighted': 0.7414954851336424, 'test_recall_weighted': 0.7071498530852106, 'test_recall_micro': 0.7071498530852106, 'test_recall_macro': 0.6579915052987269, 'test_recall_positive': 0.562753036437247, 'test_recall_negative': 0.7532299741602068, 'test_f1_weighted': 0.7199169769211087, 'test_f1_micro': 0.7071498530852106, 'test_f1_macro': 0.6388534316016112, 'test_f1_positive': 0.4818024263431542, 'test_f1_negative': 0.7959044368600683, 'test_runtime': 5.3966, 'test_samples_per_second': 189.192, 'test_steps_per_second': 11.859}


# **Muril cased training**

In [66]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [69]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['transliterated_skip_eng'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)


arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_skip_eng",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_skip_eng/log.jsonl"))

197285


Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [70]:
wandb.init(project="rh3", name="muril_skip_eng")
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.682932,0.503428,0.776736,0.503428,0.503428,0.639402,0.902834,0.375969,0.518362,0.503428,0.501216,0.467996,0.534435
2,No log,0.662398,0.790402,0.788303,0.790402,0.790402,0.584719,0.186235,0.983204,0.737364,0.790402,0.588691,0.300654,0.876728
3,0.640600,0.646615,0.789422,0.768151,0.789422,0.789422,0.639205,0.348178,0.930233,0.767118,0.789422,0.657268,0.444444,0.870091
4,0.640600,0.595779,0.774731,0.788806,0.774731,0.774731,0.721861,0.619433,0.824289,0.780415,0.774731,0.709087,0.570896,0.847278
5,0.640600,0.708400,0.770813,0.786523,0.770813,0.770813,0.719277,0.619433,0.819121,0.777065,0.770813,0.705437,0.566667,0.844208
6,0.433600,0.849662,0.798237,0.788870,0.798237,0.798237,0.698773,0.506073,0.891473,0.792247,0.798237,0.709180,0.548246,0.870113
7,0.433600,0.927381,0.785504,0.778026,0.785504,0.785504,0.688997,0.502024,0.875969,0.781142,0.785504,0.696001,0.531049,0.860952
8,0.242800,0.955070,0.792360,0.786430,0.792360,0.792360,0.701788,0.526316,0.877261,0.788976,0.792360,0.707908,0.550847,0.864968
9,0.242800,1.023754,0.789422,0.779694,0.789422,0.789422,0.687446,0.489879,0.885013,0.783355,0.789422,0.696947,0.529540,0.864353
10,0.242800,1.045396,0.787463,0.777607,0.787463,0.787463,0.684775,0.485830,0.883721,0.781340,0.787463,0.694128,0.525164,0.863091


TrainOutput(global_step=1920, training_loss=0.3744160989920298, metrics={'train_runtime': 1013.04, 'train_samples_per_second': 30.206, 'train_steps_per_second': 1.895, 'train_loss': 0.3744160989920298, 'epoch': 10.0})

In [71]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.8496617674827576, 'eval_accuracy': 0.7982370225269344, 'eval_precision_weighted': 0.7888703548806896, 'eval_recall_weighted': 0.7982370225269344, 'eval_recall_micro': 0.7982370225269344, 'eval_recall_macro': 0.6987728713554907, 'eval_recall_positive': 0.5060728744939271, 'eval_recall_negative': 0.8914728682170543, 'eval_f1_weighted': 0.7922473166488194, 'eval_f1_micro': 0.7982370225269344, 'eval_f1_macro': 0.7091795535497002, 'eval_f1_positive': 0.5482456140350876, 'eval_f1_negative': 0.8701134930643127, 'eval_runtime': 5.1385, 'eval_samples_per_second': 198.695, 'eval_steps_per_second': 12.455, 'epoch': 10.0}


In [72]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['skip_eng_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.8095677495002747, 'test_accuracy': 0.8070519098922625, 'test_precision_weighted': 0.7980881651372831, 'test_recall_weighted': 0.8070519098922625, 'test_recall_micro': 0.8070519098922625, 'test_recall_macro': 0.7101000115076003, 'test_recall_positive': 0.5222672064777328, 'test_recall_negative': 0.8979328165374677, 'test_f1_weighted': 0.801153523772847, 'test_f1_micro': 0.8070519098922625, 'test_f1_macro': 0.7214496908258723, 'test_f1_positive': 0.5670329670329671, 'test_f1_negative': 0.8758664146187776, 'test_runtime': 5.201, 'test_samples_per_second': 196.307, 'test_steps_per_second': 12.305}


## XLMR training

In [73]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [74]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['transliterated_skip_eng'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_skip_eng",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_skip_eng/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch
torch.cuda.empty_cache()

In [75]:
wandb.init(project="rh3", name="xlmr_skip_eng")
xlmr_trainer.train()

eval/accuracy,▁██▇▇██████
eval/f1_macro,▁▄▆██████▇█
eval/f1_micro,▁██▇▇██████
eval/f1_negative,▁██▇▇██████
eval/f1_positive,▅▁▅██▇▇▇▇▇▇
eval/f1_weighted,▁▇▇████████
eval/loss,▂▂▂▁▃▅▆▇██▅
eval/precision_weighted,▄█▁█▇█▄▇▅▄█
eval/recall_macro,▄▁▄██▇▆▇▆▆▇
eval/recall_micro,▁██▇▇██████
eval/recall_negative,▁█▇▆▆▇▇▇▇▇▇


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.688677,0.757101,0.574506,0.757101,0.757101,0.499354,0.000000,0.998708,0.653284,0.757101,0.430881,0.000000,0.861761
2,No log,0.578928,0.668952,0.779503,0.668952,0.668952,0.705847,0.777328,0.634367,0.692632,0.668952,0.637898,0.531856,0.743939
3,0.657700,0.564369,0.759060,0.791255,0.759060,0.759060,0.730822,0.676113,0.785530,0.769836,0.759060,0.703800,0.575862,0.831737
4,0.657700,0.624602,0.743389,0.788101,0.743389,0.743389,0.727377,0.696356,0.758398,0.757095,0.743389,0.692603,0.567657,0.817549
5,0.657700,0.700862,0.796278,0.794082,0.796278,0.796278,0.716777,0.562753,0.870801,0.795125,0.796278,0.719170,0.572016,0.866324
6,0.467400,0.849110,0.757101,0.775738,0.757101,0.757101,0.704720,0.603239,0.806202,0.764446,0.757101,0.690006,0.545788,0.834225
7,0.467400,0.993253,0.766895,0.783533,0.766895,0.766895,0.715315,0.615385,0.815245,0.773488,0.766895,0.701109,0.560886,0.841333
8,0.258400,1.781386,0.794319,0.777677,0.794319,0.794319,0.667245,0.421053,0.913437,0.780434,0.794319,0.684149,0.497608,0.870690
9,0.258400,1.674864,0.765916,0.772916,0.765916,0.765916,0.695373,0.558704,0.832041,0.769079,0.765916,0.689703,0.535922,0.843484


TrainOutput(global_step=1728, training_loss=0.4206182471028081, metrics={'train_runtime': 1544.5566, 'train_samples_per_second': 19.812, 'train_steps_per_second': 1.243, 'train_loss': 0.4206182471028081, 'epoch': 9.0})

In [76]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.7008622288703918, 'eval_accuracy': 0.7962781586679726, 'eval_precision_weighted': 0.794081859941538, 'eval_recall_weighted': 0.7962781586679726, 'eval_recall_micro': 0.7962781586679726, 'eval_recall_macro': 0.7167770350144891, 'eval_recall_positive': 0.562753036437247, 'eval_recall_negative': 0.8708010335917312, 'eval_f1_weighted': 0.7951251422270337, 'eval_f1_micro': 0.7962781586679726, 'eval_f1_macro': 0.7191701841801814, 'eval_f1_positive': 0.5720164609053499, 'eval_f1_negative': 0.8663239074550129, 'eval_runtime': 10.9174, 'eval_samples_per_second': 93.521, 'eval_steps_per_second': 5.862, 'epoch': 9.0}


In [77]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['skip_eng_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.6652987003326416, 'test_accuracy': 0.7904015670910872, 'test_precision_weighted': 0.7952071380569032, 'test_recall_weighted': 0.7904015670910872, 'test_recall_micro': 0.7904015670910872, 'test_recall_macro': 0.7253057360156503, 'test_recall_positive': 0.5991902834008097, 'test_recall_negative': 0.851421188630491, 'test_f1_weighted': 0.7925948768584851, 'test_f1_micro': 0.7904015670910872, 'test_f1_macro': 0.720352736394819, 'test_f1_positive': 0.5803921568627451, 'test_f1_negative': 0.860313315926893, 'test_runtime': 10.9178, 'test_samples_per_second': 93.517, 'test_steps_per_second': 5.862}


In [78]:
wandb.finish()
test_df.to_csv('skip_eng_muril_v2.csv',index=False, encoding='utf-8')

eval/accuracy,▆▁▆▅█▆▆█▆█
eval/f1_macro,▁▆█▇█▇█▇▇█
eval/f1_micro,▆▁▆▅█▆▆█▆█
eval/f1_negative,█▁▆▅█▆▆█▆█
eval/f1_positive,▁▇█████▇██
eval/f1_weighted,▁▃▇▆█▆▇▇▇█
eval/loss,▂▁▁▁▂▃▃█▇▂
eval/precision_weighted,▁████▇█▇▇█
eval/recall_macro,▁▇███▇█▆▇█
eval/recall_micro,▆▁▆▅█▆▆█▆█
eval/recall_negative,█▁▄▃▆▄▄▆▅▆
